In [ ]:
#Function verifyActuateablePath checks if the smooth path is feasible given robots limits
#Assumptions
# Robot follows the quadrotor model set out by Minimum  Snap  Trajectory  Generation  and  Control  for  Quadrotors - Mellinger
# Euler angles of Z-X-Y
# Polynomials of the same degree
#Inputs
# xcoeffs - the coefficients of the polynomials fit to the x portions of the points
# ycoeffs - the coefficients of the polynomials fit to the y portions of the points
# zcoeefs - the coefficients of the polynomials fit to the z portions of the points
# pcoeffs - the coefficients of the polynomials fit to the yaw angles at the points
# degree - the degree of the polynomials
# num_poly - the number of polynomials
# time_vec - vector of times at each polynomial point
#Outputs
function verifyActuateablePath(solution)
    
    time_res = 100
    red_degree_by = 2; #The two is because the we are taking the derivative twice
    red_degree = degree - red_degree_by;
    #####################3
    #create variables to hold values
    #Needed for z_b and u1
    xdd = zeros(Float64,1:num_poly*time_res);
    ydd = zeros(Float64,1:num_poly*time_res);
    zdd = zeros(Float64,1:num_poly*time_res);
    #Needed for a_dot
    xddd = zeros(Float64,1:num_poly*time_res);
    yddd = zeros(Float64,1:num_poly*time_res);
    zddd = zeros(Float64,1:num_poly*time_res);
    #Needed initializations
    z_B = 0;#zeros(1:num_poly*time_res);
    #Needed Constants
    z_w = [0,0,1];
    Jxx = 0.0036404;
    Jyy = 0.0053670;
    Jzz = 0.0030522;
    Jxy = 2.9264e-6;
    Jxz = 2.3411e-5;
    Jyz = 0.0001756;
    I = [Jxx Jxy Jxz
        Jxy Jyy Jyz
        Jxz Jyz Jzz];
    gravity = 9.8;
    u2rpm_1 = 5096839.959225280;
    u2rpm_2 = 51485858.53986801;
    u2rpm_3 = -51485858.53986803;
    u2rpm_4 = 330817430.2740964;
    u2rpms = [u2rpm_1 u2rpm_2 u2rpm_3 u2rpm_4
        u2rpm_1 -u2rpm_2 -u2rpm_3 u2rpm_4
        u2rpm_1 u2rpm_2 -u2rpm_3 -u2rpm_4
        u2rpm_1 -u2rpm_2 u2rpm_3 -u2rpm_4];
    yawdd = zeros(Float64,1:num_poly*time_res);
    yaw = zeros(Float64,1:num_poly*time_res);
    mass = 800;
    u1_vec = zeros(Float64,1:num_poly*time_res);
    u2_vec = zeros(Float64,1:num_poly*time_res);
    u3_vec = zeros(Float64,1:num_poly*time_res);
    u4_vec = zeros(Float64,1:num_poly*time_res);


    for looper = 1:(num_poly)
        #Create time vector
        t = linspace(time_vec[looper],time_vec[looper+1],time_res);
        #update coeffs for ever poly
        xddcoeffs = xcoeffs[(red_degree_by+1:degree)+degree*(looper-1)];
        yddcoeffs = ycoeffs[(red_degree_by+1:degree)+degree*(looper-1)];
        zddcoeffs = zcoeffs[(red_degree_by+1:degree)+degree*(looper-1)];
        pddcoeffs = pcoeffs[(red_degree_by+1:degree)+degree*(looper-1)];
        
        for loop=1:time_res
            cleaner = (looper-1)*time_res+loop;
            #Calculate the accelerations at every point
            for p=(3:degree)+degree*(looper-1)
                xdd[cleaner] += xcoeffs[p]*t[loop]^(p-1-degree*(looper-1))*(p-1-degree*(looper-1))*(p-2-degree*(looper-1));
                ydd[cleaner] += ycoeffs[p]*t[loop]^(p-1-degree*(looper-1))*(p-1-degree*(looper-1))*(p-2-degree*(looper-1));
                zdd[cleaner] += zcoeffs[p]*t[loop]^(p-1-degree*(looper-1))*(p-1-degree*(looper-1))*(p-2-degree*(looper-1));
                yawdd[cleaner] += pcoeffs[p]*t[loop]^(p-1-degree*(looper-1))*(p-1-degree*(looper-1))*(p-2-degree*(looper-1));
            end
            #Calculate the jerks in position at every point
            for p=(2:red_degree)
                xddd[cleaner] += xddcoeffs[p]*t[loop]^(p-2)*(p-1)*(p)*(1+p);
                yddd[cleaner] += yddcoeffs[p]*t[loop]^(p-2)*(p-1)*(p)*(1+p);
                zddd[cleaner] += zddcoeffs[p]*t[loop]^(p-2)*(p-1)*(p)*(1+p);
            end
            #Calculate the yaw at every point
            for p=(1:degree)+degree*(looper-1)
                yaw[cleaner] += pcoeffs[p]*t[loop]^(p-(1+degree*(looper-1)));
            end
            #Calculate the body centered vector pointing up relative to robot body
            z_B = [xdd[cleaner],ydd[cleaner],zdd[cleaner]]/
                sqrt((xdd[cleaner])^2 + (ydd[cleaner])^2 + (zdd[cleaner]+gravity)^2);
            #Calculate the x_c vector
            x_c = [sin(yaw[cleaner]),cos(yaw[cleaner]),0.0];
            #calculate y_B
            y_B = cross(z_B,x_c);
            #calculate x_B
            x_B = cross(y_B, z_B)
            #Calculate u1
            u1 = mass*sqrt((xdd[cleaner])^2 + (ydd[cleaner])^2 + (zdd[cleaner]+gravity)^2);
            #Calculate a_dot
            a_dot = [xddd[cleaner],yddd[cleaner],zddd[cleaner]];
            #Calculate h_w
            h_w = mass/u1*(a_dot-dot(z_B,a_dot)*z_B);
            #Calculate w_bc
            w_bc = -dot(h_w,y_B)*x_B + dot(h_w,x_B)*y_B + yaw[cleaner]*dot(z_w,z_B)*z_B;
            u2u3u4 = I*yawdd[cleaner]*z_w+cross(w_bc,I*w_bc);
            u1_vec[cleaner] = u1;
            u2_vec[cleaner] = u2u3u4[1];
            u3_vec[cleaner] = u2u3u4[2];
            u4_vec[cleaner] = u2u3u4[3];      

        end

    end

    #Calculate the needed rpms

    u_vec =[u1_vec'
        u2_vec'
        u3_vec'
        u4_vec'];

    rpms = u2rpms*u_vec
    println(size(rpms,1))
    println(size(rpms,2))
    plot((1:num_poly*time_res),rpms[1,:])
    #check that rpms are not above a certain threshold
    if(any(rpms.>9000))
        println("Motors Cannot Drive This")
    end
end